In [7]:
ROOT_DIR = "/Users/anca/Desktop/advent_code/advent_code2023/day_5/"
def load_map(path):
    print(path)
    schema = open(path).readlines()
    # print(schema)
    return [line.rstrip() for line in schema]

almanac_text = load_map(f'{ROOT_DIR}almanac.txt')
small_almanac_text = load_map(f'{ROOT_DIR}example_almanac.txt')
small_almanac_text.append("")
almanac_text.append("")
# print(small_almanac_text)

/Users/anca/Desktop/advent_code/advent_code2023/day_5/almanac.txt
/Users/anca/Desktop/advent_code/advent_code2023/day_5/example_almanac.txt


In [30]:
# destination start, source start, range
# 50 98 2 => seeds 98 -> soil 50 | seed 99 -> soil 51
from collections import *


In [9]:
def get_seeds(seeds: list[str]) -> list[int]:
    "Takes in the unformatted seeds and returns as a list of integers"
    # print(f"seeds: {seeds}")
    # print(seeds.split(' ')[1:])
    return [int(seed) for seed in seeds.split(' ')[1:]]

def format_data(map: list[str]) -> dict[int, int]:
    """
    Take a source-to-destination mapping from the almanac and return a dictionary with
    source -> destination values.
    """
    formatted_data = []
    for line in map:
        formatted_data.append([int(val) for val in line.split(' ')])
    return formatted_data


In [10]:
# get the data in a dictionaries for each table
def read_almanac(almanac):
    # get seeds
    # names_in_order = [
    #     "seed-to-soil",
    #     "soil-to-fertilizer",
    #     "fertilizer-to-water",
    #     "water-to-light",
    #     "light-to-temperatur",
    #     "temperature-to-humidity",
    #     "humidity-to-location",
    # ]
    name_to_table = {
        "seed-to-soil": [],
        "soil-to-fertilizer": [],
        "fertilizer-to-water": [],
        "water-to-light": [],
        "light-to-temperature": [],
        "temperature-to-humidity": [],
        "humidity-to-location": [],
    }

    seeds = get_seeds(almanac[0])
    # print(seeds)
    map = []
    # print(almanac[2:])

    for line in almanac[2:]:
        if line == "":
            # print(f"new table {map}")
            table_name = map[0].split(" ")[0]
            data = map[1:]
            # print(f"name {table_name}")
            name_to_table[table_name] = format_data(data)
            # print(name_to_table[table_name])
            map = []
        else:
            map.append(line)

    # print(f"table {seed_to_soil}")
    # print(f"table dic  {name_to_table['seed-to-soil']}")
    return (
        seeds,
        name_to_table["seed-to-soil"],
        name_to_table["soil-to-fertilizer"],
        name_to_table["fertilizer-to-water"],
        name_to_table["water-to-light"],
        name_to_table["light-to-temperature"],
        name_to_table["temperature-to-humidity"],
        name_to_table["humidity-to-location"],
    )


# split table and identify the headings

# identify source and destination & store data in the properly named dictionary

# test read_almanac

# check the almanc can retrieve the seed properly
print(read_almanac(small_almanac_text))
print("----------")
# print(read_almanac(almanac))
(
    seeds,
    seed_to_soil,
    soil_to_fertilizer,
    fertilizer_to_water,
    water_to_light,
    light_to_temperature,
    temperature_to_humidity,
    humidity_to_location,
) = read_almanac(small_almanac_text)

assert seeds == [79, 14, 55, 13]
assert seed_to_soil == [[50, 98, 2], [52, 50, 48]]
assert humidity_to_location == [[60, 56, 37], [56, 93, 4]]



([79, 14, 55, 13], [[50, 98, 2], [52, 50, 48]], [[0, 15, 37], [37, 52, 2], [39, 0, 15]], [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]], [[88, 18, 7], [18, 25, 70]], [[45, 77, 23], [81, 45, 19], [68, 64, 13]], [[0, 69, 1], [1, 0, 69]], [[60, 56, 37], [56, 93, 4]])
----------


In [41]:
# create an almanac conversion class. It has a read method that reads the almanac and stores the data in a dictionary
# it has methods to convert from each table to the other

import collections


class AlmanacConversion():
    def __init__(self, almanac):
        (
            self.seeds,
            self.seed_to_soil,
            self.soil_to_fertilizer,
            self.fertilizer_to_water,
            self.water_to_light,
            self.light_to_temperature,
            self.temperature_to_humidity,
            self.humidity_to_location,
        ) = read_almanac(almanac)

    def convert_from_source_to_destination(self, target_source, source_destination_table):
        """Format of each row in the source-dest table is destination, source, range"""
        for row in source_destination_table:
            destination, source, range = row
            offset = target_source - source
            if offset >= 0 and offset < range:
                return destination + offset
        # print(f"Could not find a conversion for {target_source} in {source_destination_table}. Returning the same value.")
        return target_source

    def convert_from_destination_to_source(self, target_destination, source_destination_table):
        """Format of each row in the source-dest table is destination, source, range"""
        for row in source_destination_table:
            destination, source, range = row
            offset = target_destination - destination
            if offset >= 0 and offset < range:
                return source + offset
        # print(f"Could not find a conversion for {target_source} in {source_destination_table}. Returning the same value.")
        return target_destination


    def find_location_number(self, seed):
        soil = self.convert_from_source_to_destination(seed, self.seed_to_soil)
        fertilizer = self.convert_from_source_to_destination(soil, self.soil_to_fertilizer)
        water = self.convert_from_source_to_destination(fertilizer, self.fertilizer_to_water)
        light = self.convert_from_source_to_destination(water, self.water_to_light)
        temperature = self.convert_from_source_to_destination(light, self.light_to_temperature)
        humidity = self.convert_from_source_to_destination(temperature, self.temperature_to_humidity)
        location = self.convert_from_source_to_destination(humidity, self.humidity_to_location)
        # print(f"seed: {seed} -> soil: {soil} -> fertilizer: {fertilizer} -> water: {water} -> light: {light} -> temperature: {temperature} -> humidity: {humidity} -> location: {location}")
        return location

    def find_min_location_number(self, seeds):
        return min([self.find_location_number(seed) for seed in seeds])

    def get_seed_for_location(self, location):
        humidity = self.convert_from_destination_to_source(location, self.humidity_to_location)
        temperature = self.convert_from_destination_to_source(humidity, self.temperature_to_humidity)
        light = self.convert_from_destination_to_source(temperature, self.light_to_temperature)
        water = self.convert_from_destination_to_source(light, self.water_to_light)
        fertilizer = self.convert_from_destination_to_source(water, self.fertilizer_to_water)
        soil = self.convert_from_destination_to_source(fertilizer, self.soil_to_fertilizer)
        seed = self.convert_from_destination_to_source(soil, self.seed_to_soil)
        return seed

    def find_min_location_number_for_seed_range(self):
        range_of_seeds = []

        # print(f"seeds, {self.seeds}")
        seeds = collections.deque(self.seeds.copy())
        while len(seeds) > 0:
            start = seeds.popleft()
            seed_range = seeds.popleft()
            # print(f"start: {start}, seed_range: {seed_range}")
            range_of_seeds.append([start, seed_range])
        # print(f"seeds, {self.seeds}")
        # print(f"range_of_seeds, {range_of_seeds}")
        max_location_value = self.find_min_location_number(self.seeds)
        # print(f"max_location_value: {max_location_value}")
        location = 0
        while True:
            seed = self.get_seed_for_location(location)
            if location % 10000 == 0:
                print(f"location: {location}, seed: {seed}")
            for start_seed, seed_range in range_of_seeds:
                if seed >= start_seed and seed < start_seed + seed_range:
                    print(f"seed: {seed} is in range {start_seed}, {seed_range}")
                    return location
            location += 1
        print("Could not find location for seed range")
        return -1


    def print_almanac(self):
        print(f"seeds: {self.seeds}")
        print(f"seed_to_soil: {self.seed_to_soil}")
        print(f"soil_to_fertilizer: {self.soil_to_fertilizer}")
        print(f"fertilizer_to_water: {self.fertilizer_to_water}")
        print(f"water_to_light: {self.water_to_light}")
        print(f"light_to_temperature: {self.light_to_temperature}")
        print(f"temperature_to_humidity: {self.temperature_to_humidity}")
        print(f"humidity_to_location: {self.humidity_to_location}")





# Test almanc class
example_almanac = AlmanacConversion(small_almanac_text)
# check that the conversion is correct using the print function
# almanac.print_almanac()
# check that the convert_from_source_to_destination is correct
# edge case: begging of range
assert example_almanac.convert_from_source_to_destination(98, example_almanac.seed_to_soil) == 50

# edge case: end of range
assert example_almanac.convert_from_source_to_destination(97, example_almanac.seed_to_soil) == 99

assert example_almanac.convert_from_source_to_destination(50, example_almanac.soil_to_fertilizer) == 35

# check that the find_location_number is correct
assert example_almanac.find_location_number(79) == 82
assert example_almanac.find_location_number(14) == 43
assert example_almanac.find_location_number(55) == 86
assert example_almanac.find_location_number(13) == 35

# check that the find_min_location_number is correct
assert example_almanac.find_min_location_number(example_almanac.seeds) == 35

print(example_almanac.find_min_location_number_for_seed_range())


location: 0, seed: 70
seed: 82 is in range 79, 14
46


In [42]:
almanac = AlmanacConversion(almanac_text)
location_max_bound = almanac.find_min_location_number(almanac.seeds)
print(f"Location max bound by: {location_max_bound}")
print(almanac.find_min_location_number_for_seed_range())


Location max bound by: 525792406
location: 0, seed: 770309371
location: 10000, seed: 770319371
location: 20000, seed: 770329371
location: 30000, seed: 770339371
location: 40000, seed: 770349371
location: 50000, seed: 770359371
location: 60000, seed: 770369371
location: 70000, seed: 770379371
location: 80000, seed: 770389371
location: 90000, seed: 770399371
location: 100000, seed: 770409371
location: 110000, seed: 770419371
location: 120000, seed: 770429371
location: 130000, seed: 770439371
location: 140000, seed: 770449371
location: 150000, seed: 770459371
location: 160000, seed: 770469371
location: 170000, seed: 770479371
location: 180000, seed: 770489371
location: 190000, seed: 770499371
location: 200000, seed: 770509371
location: 210000, seed: 770519371
location: 220000, seed: 770529371
location: 230000, seed: 770539371
location: 240000, seed: 770549371
location: 250000, seed: 770559371
location: 260000, seed: 770569371
location: 270000, seed: 770579371
location: 280000, seed: 77058

In [28]:
my_list = [1,2,3,4]
print(my_list.pop())

4
